# Checking on error estimation by comparing SDSS to BOSS results

In [13]:
# imports
# imports
import numpy as np
import os, sys

from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u

from linetools.utils import dv_from_z

from dla_cnn.io import load_ml_dr7, load_ml_dr12, load_garnett16
from dla_cnn.catalogs import match_boss_catalogs

sys.path.append(os.path.abspath("../Analysis/py"))
import defs as analy_defs

## Load em up

### BOSS

In [3]:
# Load BOSS ML
_, dr12_abs = load_ml_dr12()

In [9]:
dlas = dr12_abs['NHI'] >= 20.3
dr12_dla = dr12_abs[dlas]

### SDSS

In [4]:
_, dr7_dlas = load_ml_dr7()

Loading SDSSDR7 model.  Please be patient..
Looping on sightlines..


## Match em

In [10]:
dr12_dla_coords = SkyCoord(ra=dr12_dla['RA'], dec=dr12_dla['DEC'], unit='deg')

In [14]:
idx_dr7, idx_dr12, d2d, d3d = dr12_dla_coords.search_around_sky(dr7_dlas.coord, 1*u.arcsec)

In [15]:
dr12_to_dr7 = np.zeros(len(dr12_dla)).astype(int) -1

In [19]:
dztoler = 0.015
dr7_zabs = dr7_dlas.zabs
dr7_NHI = dr7_dlas.NHI
dNHI = []
for kk,idx in enumerate(idx_dr12):
    dz = np.abs(dr12_dla['zabs'][idx] - dr7_zabs[idx_dr7[kk]])
    if dz < dztoler:
        dr12_to_dr7[idx] = idx_dr7[kk]
        dNHI.append(np.abs(dr12_dla['NHI'][idx] - dr7_NHI[idx_dr7[kk]]))

In [21]:
np.median(dNHI)

0.10155322503647035